## Генерация заголовков с помощью LLaMa 2

In [ ]:
import torch
from peft import PeftModel
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

token = "{your_hf_token}"

BASE_MODEL = "meta-llama/Llama-2-7b-hf"
ADAPTER_MODEL = "pineforest-ai/news-headings-rus"

# Инициализируем основную модель
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL, use_auth_token=token)

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=False,
    device_map="auto",
)

# Добавим к предобученной LLaMa2 адаптеры, обученные на задаче генерации заголовков
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, torch_dtype=torch.float16)

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [ ]:
model = model.eval()
model = torch.compile(model)

In [ ]:
INSTRUCTION = "Сгенерируй заголовок к данному тексту."

In [ ]:
def generate_response(prompt, model):
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to(DEVICE)
 
    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.75,
        repetition_penalty=1.1,
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=128,
        )

In [ ]:
def generate_heading(text, model):
    prompt = f"{INSTRUCTION}\nТекст:\n{text}\nЗаголовок: "
    response = generate_response(prompt, model)
    decoded_output = tokenizer.decode(response.sequences[0], skip_special_tokens=True)
    decoded_output_lines = decoded_output.split("\n")
    for line in decoded_output_lines:
        if "Заголовок:" in line:
            heading = line
            break
    print(heading)

In [ ]:
text = "Модель ИИ для автоматического определения типа дыхания при терапии легких создали специалисты ВолгГТУ. "\
       "По их словам, разработка, не имеющая аналогов, может работать в домашних условиях с обычной камерой, "\
       "что расширяет возможности реабилитации. Результаты опубликованы в журнале Algorithms. По словам ученых, "\
       "специальные дыхательные тренировки являются одним из важнейших элементов реабилитации после ряда болезней, "\
       "также они могут быть эффективным способом снижения стресса и общего укрепления организма. "\
       "Для этого во всем мире активно разрабатываются технические комплексы, способные контролировать "\
       "качество тренировок с точностью, превосходящей возможности тренера-человека. Коллектив Волгоградской "\
       "инновационной лаборатории анализа и управления данными (V.I.S.D.O.M.) Волгоградского государственного "\
       "технического университета (ВолгГТУ) кафедры программного обеспечения автоматизированных систем совместно "\
       "с коллегами создал модель для реабилитации людей с патологиями дыхательной системы. По словам авторов, "\
       "разработка дает возможность организовать эффективные тренировки даже в домашних условиях."

In [ ]:
generate_heading(text, model)